## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.30"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


In [ ]:
import Python_scripts.Feature_functions.compute_motion_features
importlib.reload(Python_scripts.Feature_functions.compute_motion_features)
from Python_scripts.Feature_functions.compute_motion_features import compute_motion_features

total_distance, average_velocity, cumulative_distance = compute_motion_features(
    conn, 49, bodypart_x='head_x_norm', bodypart_y='head_y_norm', time_limit=480.0)

## Insert distance, velocity, cumulative distance

In [ ]:
from Python_scripts.Insert_to_featuretable.insert_motion_features import insert_motion_features

query = "SELECT id FROM dlc_table WHERE head_x_norm IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()

insert_motion_features(id_list, conn)


## Insert acceleration and jerk outliers

In [ ]:
from Python_scripts.Insert_to_featuretable.insert_motion_outliers import insert_motion_outliers

query = "SELECT id FROM dlc_table WHERE head_x IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()

insert_motion_outliers(id_list, conn, bodypart_x='head_x_norm', bodypart_y='head_y_norm', time_limit=1200.0)


## Insert curvature

In [ ]:
from Python_scripts.Insert_to_featuretable.insert_trajectory_curvature import insert_trajectory_curvature

query = "SELECT id FROM dlc_table WHERE head_x IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()

insert_trajectory_curvature(id_list, conn)
